# 按照ip将数据文件中被攻击的IP流量包数据

In [1]:
# import libs
import numpy as np
import pandas as pd
import os

In [2]:
# def function

# 获取csv文件的列名
def getcols(filename):
    data = pd.read_csv(filename, nrows = 2)
    #print(data.columns)
    return data.columns

# 获取文件行数
def getrows(filename):
    cols = getcols(filename)
    col = cols[0]
    data = pd.read_csv(filename, usecols=[col])
    return data.shape[0]

# 数据预处理- step1: 删除无法用于计算的列与含空值、负值、inf值的行
def preprocess(data, cols, labelnum, label):
    # print(data.columns)
    
        # 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
    dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1']
    # 删除含有大量负数的列
    drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
           ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
    # 删除数据中可能出现的非数值型数据列
    drop_ifexist = ['External IP','SimillarHTTP', ' Inbound','Unnamed: 0', 'Unnamed: 0.1']
    
    # 如果有list中的columns则删除
    data.drop(data.columns[data.columns.isin(dropset+drop_nega+drop_ifexist)], axis = 1, inplace=True)
    
    # 将标签替换为数字
    data.replace(to_replace=label, value=labelnum, inplace=True)
    
    # 将所有数据类型转成float型
    data = data.astype('float')
    
    data.dropna(how='any', inplace=True)
#    print(len(data))
    data.reset_index(inplace=True, drop=True)
    # 将所有数据类型转成float型
    data = data.astype('float')
    
    # 删除数据中inf的值 (需要连续执行两次，不知道原因，但连续执行后就可删除数据中的inf)
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
   # print(idx)
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)

   # print(len(data))
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)

   # print(len(data))

    # 删除数据中含负数的数据条数
    idx_tuple = np.where(data.min(axis=1).values < 0)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    return data

In [3]:
# 从文件夹'newdata/'中读取数据文件
filedir = '/mnt/hgfs/linuxfile/2017/'
files = os.listdir(filedir)

In [4]:
files

['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'Monday-WorkingHours.pcap_ISCX.csv',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
 'Tuesday-WorkingHours.pcap_ISCX.csv',
 'Wednesday-workingHours.pcap_ISCX.csv']

In [5]:
# 受害者IP
DDos_victim_IP = ['192.168.10.50'] # 128834
PortScan_victim_IP = ['192.168.10.50'] # 160716
Botnet_victim_IP = ['192.168.10.15', '192.168.10.9', '192.168.10.14', '192.168.10.5', '192.168.10.8'] # 16029 
WebAttack_victim_IP = ['192.168.10.50'] # 4230
Infiltration_victim_IP = ['192.168.10.8', '192.168.10.25'] # 15943
BruteForce_victim_IP = ['192.168.10.50'] # 4188
DDos2_victim_IP = ['192.168.10.50','205.174.165.66','192.168.10.51'] # 23926
ips = [DDos_victim_IP, PortScan_victim_IP, Botnet_victim_IP, WebAttack_victim_IP, Infiltration_victim_IP, BruteForce_victim_IP, DDos2_victim_IP]

In [6]:
# i = 6
# data = pd.read_csv(filedir+files[i])
# data[data[' Destination IP'].isin(ips[i])]

In [7]:
# 从文件夹'newdata/'中读取数据文件
filedir = '/mnt/hgfs/linuxfile/2019/0112/'
files = os.listdir(filedir)

In [8]:
files

['DrDoS_DNS.csv',
 'DrDoS_LDAP.csv',
 'DrDoS_MSSQL.csv',
 'DrDoS_NetBIOS.csv',
 'DrDoS_NTP.csv',
 'DrDoS_SNMP.csv',
 'DrDoS_SSDP.csv',
 'DrDoS_UDP.csv',
 'Syn.csv',
 'TFTP.csv']

In [9]:
# 受害者IP
DNS_victim_IP = ['192.168.50.1']

In [10]:
for i in range(10):
    data = pd.read_csv(filedir+files[i], usecols=[' Destination IP'])
    L = len(data[data[' Destination IP'].isin(DNS_victim_IP)])
    print(filedir+files[i], L)

/mnt/hgfs/linuxfile/2019/0112/DrDoS_DNS.csv 5069066
/mnt/hgfs/linuxfile/2019/0112/DrDoS_LDAP.csv 2179205
/mnt/hgfs/linuxfile/2019/0112/DrDoS_MSSQL.csv 4521636
/mnt/hgfs/linuxfile/2019/0112/DrDoS_NetBIOS.csv 4092295
/mnt/hgfs/linuxfile/2019/0112/DrDoS_NTP.csv 1200627
/mnt/hgfs/linuxfile/2019/0112/DrDoS_SNMP.csv 5158515
/mnt/hgfs/linuxfile/2019/0112/DrDoS_SSDP.csv 2609649
/mnt/hgfs/linuxfile/2019/0112/DrDoS_UDP.csv 3132763
/mnt/hgfs/linuxfile/2019/0112/Syn.csv 1582112
/mnt/hgfs/linuxfile/2019/0112/TFTP.csv 20078140


### 各攻击与受害者ip相关的数据
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_DNS.csv 5069066
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_LDAP.csv 2179205
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_MSSQL.csv 4521636
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_NetBIOS.csv 4092295
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_NTP.csv 1200627
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_SNMP.csv 5158515
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_SSDP.csv 2609649
* /mnt/hgfs/linuxfile/2019/0112/DrDoS_UDP.csv 3132763
* /mnt/hgfs/linuxfile/2019/0112/Syn.csv 1582112
* /mnt/hgfs/linuxfile/2019/0112/TFTP.csv 20078140

In [11]:
['DrDoS_DNS.csv',
 'DrDoS_LDAP.csv',
 'DrDoS_MSSQL.csv',
 'DrDoS_NetBIOS.csv',
 'DrDoS_NTP.csv',
 'DrDoS_SNMP.csv',
 'DrDoS_SSDP.csv',
 'DrDoS_UDP.csv',
 'Syn.csv',
 'TFTP.csv']

['DrDoS_DNS.csv',
 'DrDoS_LDAP.csv',
 'DrDoS_MSSQL.csv',
 'DrDoS_NetBIOS.csv',
 'DrDoS_NTP.csv',
 'DrDoS_SNMP.csv',
 'DrDoS_SSDP.csv',
 'DrDoS_UDP.csv',
 'Syn.csv',
 'TFTP.csv']

In [12]:
len(data[data[' Destination IP'].isin(DNS_victim_IP)])

20078140

In [ ]:
filedir = '/mnt/hgfs/linuxfile/2017/'
files = os.listdir(filedir)